# Create a DB with patients ID / CLINICAL DATA / Path images in time

## IMPORT

In [1]:
# What version of Python do you have?
import sys
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

import gc
import copy
import cv2

from pathlib import Path

In [2]:
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Keras Version: {np.__version__}")

Python 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
Pandas 1.3.5
Keras Version: 1.22.3


In [3]:
!nvidia-smi

Mon Jun 13 15:09:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.95       Driver Version: 512.95       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| 61%   50C    P2    95W / 350W |   3950MiB / 12288MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### UTILS PATHS

In [4]:
databasePath = Path('../CHEST-RAY-ONLY-LABELED-ICU-ONE-PER-PATIENT-OLDEST/newDB.csv')
databasePathIMAGES = Path('../COVID-19-CHEST-ONLY/manifest-1645033969298/metadata.csv')


### RANDOM NUMBER

In [5]:
random_seed = 42

### GLOBAL VARIABLES

In [6]:
IMAGE_SIZE = [224,224]
CLASS_NAMES = ['MILD', 'SEVERE']

# REED CSVs

In [7]:
# # read in all our data
# COVID19 = pd.read_csv(databasePath)
# COVID19 = COVID19.drop(['covid19_statuses'], axis='columns')

# COVID19 = pd.DataFrame(COVID19[['to_patient_id','is_icu','FilePaths']])
# print(COVID19.shape)

In [8]:
COVID19_data = pd.read_csv(databasePath)
COVID19_IMGs = pd.read_csv(databasePathIMAGES)

In [9]:
COVID19_data.head()

,Unnamed: 0,to_patient_id,covid19_statuses,last.status,age.splits,gender_concept_name,visit_start_datetime,visit_concept_name,is_icu,was_ventilated,...,30341-2_Erythrocyte sedimentation rate,2345-7_Glucose [Mass/volume] in Serum or Plasma,13457-7_Cholesterol in LDL [Mass/volume] in Serum or Plasma by calculation,13458-5_Cholesterol in VLDL [Mass/volume] in Serum or Plasma by calculation,2571-8_Triglyceride [Mass/volume] in Serum or Plasma,2085-9_Cholesterol in HDL [Mass/volume] in Serum or Plasma,therapeutic.exnox.Boolean,therapeutic.heparin.Boolean,Other.anticoagulation.therapy,FilePaths
0,0,A917122,positive,discharged,"(74,90]",FEMALE,1/1/1901,Inpatient Visit,MILD,No,...,39.0,128.0,NaN,NaN,NaN,NaN,False,True,apixaban,/MILD/0_A917122.png
1,1,A387558,positive,deceased,"(74,90]",MALE,1/1/1901,Inpatient Visit,SEVERE,No,...,NaN,199.0,NaN,NaN,NaN,NaN,True,True,apixaban,/SEVERE/1_A387558.png
2,2,A968238,positive,deceased,"(74,90]",MALE,1/5/1901,Inpatient Visit,SEVERE,Yes,...,54.0,134.0,NaN,NaN,NaN,NaN,False,True,not documented,/SEVERE/2_A968238.png
3,5,A642300,positive,deceased,"(74,90]",FEMALE,12/31/1900,Inpatient Visit,SEVERE,Yes,...,NaN,246.0,NaN,NaN,NaN,NaN,False,True,not documented,/SEVERE/3_A642300.png
4,6,A205701,positive,discharged,"(59,74]",FEMALE,1/1/1901,Inpatient Visit,MILD,No,...,32.0,106.0,NaN,NaN,NaN,NaN,False,False,not documented,/MILD/4_A205701.png


In [10]:

COVID19_IMGs.head()

,Series UID,Collection,3rd Party Analysis,Data Description URI,Subject ID,Study UID,Study Description,Study Date,Series Description,Manufacturer,Modality,SOP Class Name,SOP Class UID,Number of Images,File Size,File Location,Download Timestamp
1.3.6.1.4.1.14519.5.2.1.99.1071.22834706453880258048743235235639,COVID-19-NY-SBU,NO,https://doi.org/10.7937/TCIA.BBAG2923,A000801,1.3.6.1.4.1.14519.5.2.1.99.1071.21209260917602...,CHEST AP PORT,01-11-1901,AP,CARESTREAM HEALTH,CR,Computed Radiography Image Storage,1.2.840.10008.5.1.4.1.1.1,1,15,25 MB,.\COVID-19-NY-SBU\A000801\01-11-1901-NA-CHEST ...,2022-02-16T18:58:09.295
1.3.6.1.4.1.14519.5.2.1.99.1071.11932427966520600842104275399857,COVID-19-NY-SBU,NO,https://doi.org/10.7937/TCIA.BBAG2923,A000801,1.3.6.1.4.1.14519.5.2.1.99.1071.10521361127718...,CHEST AP PORT,01-10-1901,AP,CARESTREAM HEALTH,CR,Computed Radiography Image Storage,1.2.840.10008.5.1.4.1.1.1,1,15,25 MB,.\COVID-19-NY-SBU\A000801\01-10-1901-NA-CHEST ...,2022-02-16T18:58:11.356
1.3.6.1.4.1.14519.5.2.1.99.1071.30459114875858176018345224902435,COVID-19-NY-SBU,NO,https://doi.org/10.7937/TCIA.BBAG2923,A000801,1.3.6.1.4.1.14519.5.2.1.99.1071.10521361127718...,CHEST AP PORT,01-10-1901,AP,CARESTREAM HEALTH,CR,Computed Radiography Image Storage,1.2.840.10008.5.1.4.1.1.1,1,15,25 MB,.\COVID-19-NY-SBU\A000801\01-10-1901-NA-CHEST ...,2022-02-16T18:58:11.52
1.3.6.1.4.1.14519.5.2.1.99.1071.96343567154278993525506303057115,COVID-19-NY-SBU,NO,https://doi.org/10.7937/TCIA.BBAG2923,A000801,1.3.6.1.4.1.14519.5.2.1.99.1071.21209260917602...,CHEST AP PORT,01-11-1901,AP,CARESTREAM HEALTH,CR,Computed Radiography Image Storage,1.2.840.10008.5.1.4.1.1.1,1,15,25 MB,.\COVID-19-NY-SBU\A000801\01-11-1901-NA-CHEST ...,2022-02-16T18:58:16.463
1.3.6.1.4.1.14519.5.2.1.99.1071.47315006382151016305082180949670,COVID-19-NY-SBU,NO,https://doi.org/10.7937/TCIA.BBAG2923,A000801,1.3.6.1.4.1.14519.5.2.1.99.1071.23281651418629...,CHEST AP VIEWONLY,01-14-1901,AP,CARESTREAM HEALTH,CR,Computed Radiography Image Storage,1.2.840.10008.5.1.4.1.1.1,1,15,28 MB,.\COVID-19-NY-SBU\A000801\01-14-1901-NA-CHEST ...,2022-02-16T18:58:18.853


In [11]:
COVID19_data = COVID19_data.reset_index()
COVID19_IMGs = COVID19_IMGs.reset_index()

In [12]:
COVID19_data.head()


,index,Unnamed: 0,to_patient_id,covid19_statuses,last.status,age.splits,gender_concept_name,visit_start_datetime,visit_concept_name,is_icu,...,30341-2_Erythrocyte sedimentation rate,2345-7_Glucose [Mass/volume] in Serum or Plasma,13457-7_Cholesterol in LDL [Mass/volume] in Serum or Plasma by calculation,13458-5_Cholesterol in VLDL [Mass/volume] in Serum or Plasma by calculation,2571-8_Triglyceride [Mass/volume] in Serum or Plasma,2085-9_Cholesterol in HDL [Mass/volume] in Serum or Plasma,therapeutic.exnox.Boolean,therapeutic.heparin.Boolean,Other.anticoagulation.therapy,FilePaths
0,0,0,A917122,positive,discharged,"(74,90]",FEMALE,1/1/1901,Inpatient Visit,MILD,...,39.0,128.0,NaN,NaN,NaN,NaN,False,True,apixaban,/MILD/0_A917122.png
1,1,1,A387558,positive,deceased,"(74,90]",MALE,1/1/1901,Inpatient Visit,SEVERE,...,NaN,199.0,NaN,NaN,NaN,NaN,True,True,apixaban,/SEVERE/1_A387558.png
2,2,2,A968238,positive,deceased,"(74,90]",MALE,1/5/1901,Inpatient Visit,SEVERE,...,54.0,134.0,NaN,NaN,NaN,NaN,False,True,not documented,/SEVERE/2_A968238.png
3,3,5,A642300,positive,deceased,"(74,90]",FEMALE,12/31/1900,Inpatient Visit,SEVERE,...,NaN,246.0,NaN,NaN,NaN,NaN,False,True,not documented,/SEVERE/3_A642300.png
4,4,6,A205701,positive,discharged,"(59,74]",FEMALE,1/1/1901,Inpatient Visit,MILD,...,32.0,106.0,NaN,NaN,NaN,NaN,False,False,not documented,/MILD/4_A205701.png


In [13]:
date_Parsed_COVID19 = pd.to_datetime(COVID19_IMGs['Study Description'],infer_datetime_format=True, exact=True)
COVID19_IMGs['visit_start_datetime'] = date_Parsed_COVID19

In [14]:
COVID19_IMGs.head()


,index,Series UID,Collection,3rd Party Analysis,Data Description URI,Subject ID,Study UID,Study Description,Study Date,Series Description,Manufacturer,Modality,SOP Class Name,SOP Class UID,Number of Images,File Size,File Location,Download Timestamp,visit_start_datetime
0,1.3.6.1.4.1.14519.5.2.1.99.1071.22834706453880...,COVID-19-NY-SBU,NO,https://doi.org/10.7937/TCIA.BBAG2923,A000801,1.3.6.1.4.1.14519.5.2.1.99.1071.21209260917602...,CHEST AP PORT,01-11-1901,AP,CARESTREAM HEALTH,CR,Computed Radiography Image Storage,1.2.840.10008.5.1.4.1.1.1,1,15,25 MB,.\COVID-19-NY-SBU\A000801\01-11-1901-NA-CHEST ...,2022-02-16T18:58:09.295,1901-01-11
1,1.3.6.1.4.1.14519.5.2.1.99.1071.11932427966520...,COVID-19-NY-SBU,NO,https://doi.org/10.7937/TCIA.BBAG2923,A000801,1.3.6.1.4.1.14519.5.2.1.99.1071.10521361127718...,CHEST AP PORT,01-10-1901,AP,CARESTREAM HEALTH,CR,Computed Radiography Image Storage,1.2.840.10008.5.1.4.1.1.1,1,15,25 MB,.\COVID-19-NY-SBU\A000801\01-10-1901-NA-CHEST ...,2022-02-16T18:58:11.356,1901-01-10
2,1.3.6.1.4.1.14519.5.2.1.99.1071.30459114875858...,COVID-19-NY-SBU,NO,https://doi.org/10.7937/TCIA.BBAG2923,A000801,1.3.6.1.4.1.14519.5.2.1.99.1071.10521361127718...,CHEST AP PORT,01-10-1901,AP,CARESTREAM HEALTH,CR,Computed Radiography Image Storage,1.2.840.10008.5.1.4.1.1.1,1,15,25 MB,.\COVID-19-NY-SBU\A000801\01-10-1901-NA-CHEST ...,2022-02-16T18:58:11.52,1901-01-10
3,1.3.6.1.4.1.14519.5.2.1.99.1071.96343567154278...,COVID-19-NY-SBU,NO,https://doi.org/10.7937/TCIA.BBAG2923,A000801,1.3.6.1.4.1.14519.5.2.1.99.1071.21209260917602...,CHEST AP PORT,01-11-1901,AP,CARESTREAM HEALTH,CR,Computed Radiography Image Storage,1.2.840.10008.5.1.4.1.1.1,1,15,25 MB,.\COVID-19-NY-SBU\A000801\01-11-1901-NA-CHEST ...,2022-02-16T18:58:16.463,1901-01-11
4,1.3.6.1.4.1.14519.5.2.1.99.1071.47315006382151...,COVID-19-NY-SBU,NO,https://doi.org/10.7937/TCIA.BBAG2923,A000801,1.3.6.1.4.1.14519.5.2.1.99.1071.23281651418629...,CHEST AP VIEWONLY,01-14-1901,AP,CARESTREAM HEALTH,CR,Computed Radiography Image Storage,1.2.840.10008.5.1.4.1.1.1,1,15,28 MB,.\COVID-19-NY-SBU\A000801\01-14-1901-NA-CHEST ...,2022-02-16T18:58:18.853,1901-01-14


In [15]:
patiensNames = COVID19_IMGs['Data Description URI'].unique()

In [16]:
print(patiensNames)

['A000801' 'A000936' 'A001942' ... 'A998860' 'A999046' 'A999787']


In [17]:
def changePath(x):
    x = Path(x)
    x = x / '1-1.dcm'
    return(x)

In [18]:
DB = pd.DataFrame()

DB['number_images_in_time'] = np.nan
DB['images_paths_in_time'] = []
DB['images_timestamp'] = []

for patient in tqdm(patiensNames):
    patientImages = COVID19_IMGs.loc[COVID19_IMGs['Data Description URI']==patient]
    patientImages = patientImages[patientImages['Study Date'] != 'PA']
    # # patientImages = patientImages['FilePahts']
    patientData = COVID19_data.loc[COVID19_data['to_patient_id']==patient]
    patientImages = patientImages.sort_values('visit_start_datetime')

    patientImages['File Location'] = patientImages['File Location'].apply(changePath)
    
    if (patientData['covid19_statuses'].empty):
        print("DATA Not present")
    else:
        # COVID19_data['visit_start_datetime']
        # patientData['orderedImages'] = patientImages['File Location']

        patientDates = patientImages['visit_start_datetime'].to_list()
        patientImages = patientImages['File Location'].to_list()

        numberImages = len(patientImages)


        print(len(COVID19_data))
        # print(patientDates.dtype)

        # COVID19_data['images_paths_in_time'] = COVID19_data['images_paths_in_time'] .astype('object')
        # COVID19_data['images_paths_in_time'] = [list() for _ in range(len(COVID19_data))]
        # COVID19_data['images_timestamp'] = [list() for _ in range(len(COVID19_data))]

        # COVID19_data.at[patientData.index[0], 'number_images_in_time'] += numberImages
        # COVID19_data.at[patientData.index[0], 'images_paths_in_time'] += patientImages
        # COVID19_data.at[patientData.index[0], 'images_timestamp'] = patientDates
        
        DB = pd.concat([DB, patientData], ignore_index=True)
        # DB['number_images_in_time'] = DB['number_images_in_time'].astype('int64')
        # DB['images_paths_in_time'] = DB['images_paths_in_time'].astype('object')
        # DB['images_timestamp'] = DB['images_timestamp'].astype('object')
        # DB['number_images_in_time'][-1] = numberImages
        # DB['images_paths_in_time'][-1] = patientImages
        # DB['images_timestamp'][-1] = patientDates
        DB.iat[-1, DB.columns.get_loc('images_paths_in_time')] = patientImages
        DB.iat[-1, DB.columns.get_loc('images_timestamp')] = patientDates
        DB.iat[-1, DB.columns.get_loc('number_images_in_time')] = numberImages

        # print(DB)
        # break
        # patientData['images_paths_time'] = patientImages
        # patientData['images_timestamp'] = patientDates


        # print(patientData['images_paths_time'])
        # print(patientImages)
        # DB = pd.concat([DB, patientData], ignore_index=True)

  0%|          | 0/1330 [00:00<?, ?it/s]

1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279


  3%|▎         | 36/1330 [00:00<00:03, 357.63it/s]

1279
1279
1279
1279
DATA Not present
1279
1279
1279
DATA Not present
1279
1279
1279
1279
1279
1279
DATA Not present
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279


  5%|▌         | 72/1330 [00:00<00:03, 347.51it/s]

...

1279
1279
1279
1279
1279
1279
1279
1279
1279
DATA Not present
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279


 98%|█████████▊| 1309/1330 [00:03<00:00, 312.04it/s]

1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279
1279


100%|██████████| 1330/1330 [00:04<00:00, 331.14it/s]

1279
1279
1279
1279
1279


In [39]:
DB.to_csv('./DB_in_time.csv')